In [1]:
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
df_label = pd.read_csv('./Platin_Data/patient.csv', sep = '\t', index_col = 0)

In [3]:
df_label

,PLATINUM_STATUS
ID,
TCGA-61-1910,Sensitive
TCGA-24-0982,Resistant
TCGA-36-1580,Resistant
TCGA-25-1321,Sensitive
TCGA-30-1892,Resistant
...,...
TCGA-04-1347,Sensitive
TCGA-24-0968,Sensitive
TCGA-30-1891,Resistant


In [4]:
df_label.loc[df_label['PLATINUM_STATUS'] == 'Sensitive', 'label'] = 0
df_label.loc[df_label['PLATINUM_STATUS'] == 'Resistant', 'label'] = 1

In [5]:
df_label

,PLATINUM_STATUS,label
ID,,
TCGA-61-1910,Sensitive,0.0
TCGA-24-0982,Resistant,1.0
TCGA-36-1580,Resistant,1.0
TCGA-25-1321,Sensitive,0.0
TCGA-30-1892,Resistant,1.0
...,...,...
TCGA-04-1347,Sensitive,0.0
TCGA-24-0968,Sensitive,0.0
TCGA-30-1891,Resistant,1.0


In [6]:
df_data = pd.read_csv('./Platin_Data/final_minmax.csv', sep = '\t', index_col = 0)

In [7]:
df_data

,TCGA-61-1910,TCGA-24-0982,TCGA-36-1580,TCGA-25-1321,TCGA-30-1892,TCGA-61-1741,TCGA-13-0727,TCGA-04-1542,TCGA-61-1918,TCGA-25-1316,...,TCGA-04-1530,TCGA-13-0897,TCGA-09-1666,TCGA-25-2398,TCGA-20-1682,TCGA-04-1347,TCGA-24-0968,TCGA-30-1891,TCGA-09-0366,TCGA-13-0884
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000015413,6.708431e-06,2.291457e-06,6.538119e-06,1.185424e-06,0.000006,5.920554e-06,5.076250e-06,1.361885e-06,7.252694e-06,8.269170e-06,...,7.925408e-07,6.625105e-07,1.583949e-06,2.893102e-06,2.396280e-06,1.104610e-06,2.779914e-06,3.458445e-06,1.007917e-05,8.070000e-07
ENSG00000048545,9.127873e-08,9.544551e-08,4.549161e-07,3.780364e-08,0.000001,2.581000e-06,6.386642e-07,8.686208e-08,9.026001e-08,0.000000e+00,...,1.284536e-06,1.859240e-06,3.402964e-07,6.010221e-06,4.528492e-07,0.000000e+00,1.818514e-07,6.617471e-07,5.877554e-07,2.352982e-07
ENSG00000069482,1.155713e-05,5.247303e-06,2.357835e-05,5.038372e-06,0.000004,8.404273e-06,4.965306e-06,1.360269e-05,1.924740e-05,4.686161e-05,...,7.925912e-07,1.942936e-05,2.834615e-05,3.583541e-05,3.621275e-06,4.590562e-06,3.150770e-06,1.494432e-05,3.549534e-06,9.930649e-06
ENSG00000072657,4.240538e-08,6.651171e-08,1.451631e-07,5.109074e-08,0.000003,3.740853e-07,6.000000e-09,2.934802e-08,3.430809e-08,1.131410e-07,...,1.081000e-06,0.000000e+00,2.155794e-08,7.660000e-07,1.912548e-08,7.053000e-09,1.290000e-07,9.316000e-09,1.551441e-09,5.000000e-09
ENSG00000078399,5.295986e-07,1.199844e-06,2.248395e-06,7.311215e-08,0.000002,7.941253e-07,5.764151e-05,0.000000e+00,1.745627e-06,1.495767e-03,...,9.201074e-07,3.105434e-06,1.480798e-06,1.855722e-05,1.068487e-05,4.844650e-07,2.461901e-06,2.133000e-06,2.962571e-05,4.550662e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278698,0.000000e+00,3.585463e-06,0.000000e+00,8.520686e-06,0.000000,4.407119e-06,0.000000e+00,0.000000e+00,3.390667e-06,5.786538e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.423000e-05,0.000000e+00,2.732539e-06,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000279834,1.064800e-05,4.639179e-07,0.000000e+00,2.572452e-06,0.000000,2.280923e-06,6.898357e-07,0.000000e+00,7.896843e-06,0.000000e+00,...,2.774913e-06,0.000000e+00,8.270000e-07,0.000000e+00,4.402195e-06,0.000000e+00,0.000000e+00,0.000000e+00,1.071306e-06,7.624528e-07
ENSG00000279970,1.919653e-05,4.132642e-06,1.188083e-05,3.320444e-05,0.000006,7.982371e-06,2.633635e-06,1.182026e-05,2.344871e-05,9.942297e-06,...,1.765664e-06,2.091000e-06,1.157700e-05,1.304427e-06,8.179200e-05,1.033000e-06,9.448659e-06,0.000000e+00,4.499000e-05,2.425725e-06


In [8]:
df_data_mod = df_data.T

In [9]:
df_data_mod

ID,ENSG00000015413,ENSG00000048545,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,6.708431e-06,9.127873e-08,0.000012,4.240538e-08,5.295986e-07,0.000000e+00,0.000000e+00,0.000000e+00,5.207304e-07,0.000027,...,0.000000,1.096190e-05,0.000000,0.000019,0.000004,0.000000,1.064800e-05,0.000019,0.000009,0.000000e+00
TCGA-24-0982,2.291457e-06,9.544551e-08,0.000005,6.651171e-08,1.199844e-06,9.374258e-08,0.000000e+00,8.546588e-08,1.089000e-06,0.000003,...,0.000329,3.438690e-06,0.000061,0.000002,0.000056,0.000004,4.639179e-07,0.000004,0.000005,0.000000e+00
TCGA-36-1580,6.538119e-06,4.549161e-07,0.000024,1.451631e-07,2.248395e-06,3.309626e-08,4.300000e-08,7.015486e-07,1.249552e-06,0.000017,...,0.000067,4.856182e-06,0.000015,0.000005,0.000081,0.000000,0.000000e+00,0.000012,0.000269,0.000000e+00
TCGA-25-1321,1.185424e-06,3.780364e-08,0.000005,5.109074e-08,7.311215e-08,2.648546e-06,1.286245e-08,0.000000e+00,1.653421e-06,0.000004,...,0.000042,2.724000e-06,0.000000,0.000002,0.000022,0.000009,2.572452e-06,0.000033,0.000024,0.000000e+00
TCGA-30-1892,5.760662e-06,1.175743e-06,0.000004,2.743489e-06,2.463374e-06,1.154765e-06,0.000000e+00,1.140543e-06,5.775835e-06,0.000012,...,0.000022,2.353300e-06,0.000058,0.000000,0.000077,0.000000,0.000000e+00,0.000006,0.000039,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,1.104610e-06,0.000000e+00,0.000005,7.053000e-09,4.844650e-07,8.200000e-08,1.420513e-08,1.682306e-07,8.733131e-07,0.000010,...,0.000000,0.000000e+00,0.000000,0.000044,0.000008,0.000000,0.000000e+00,0.000001,0.000066,0.000000e+00
TCGA-24-0968,2.779914e-06,1.818514e-07,0.000003,1.290000e-07,2.461901e-06,0.000000e+00,0.000000e+00,0.000000e+00,4.080569e-06,0.000003,...,0.000202,6.988482e-06,0.000058,0.000036,0.000423,0.000003,0.000000e+00,0.000009,0.000081,0.000000e+00
TCGA-30-1891,3.458445e-06,6.617471e-07,0.000015,9.316000e-09,2.133000e-06,0.000000e+00,1.250863e-08,3.884533e-06,0.000000e+00,0.000002,...,0.000916,0.000000e+00,0.000051,0.000000,0.001269,0.000000,0.000000e+00,0.000000,0.001556,0.000000e+00
TCGA-09-0366,1.007917e-05,5.877554e-07,0.000004,1.551441e-09,2.962571e-05,4.810573e-07,2.500000e-08,1.151283e-07,5.518561e-06,0.000013,...,0.000004,9.705448e-06,0.000002,0.000027,0.000067,0.000000,1.071306e-06,0.000045,0.000011,2.857222e-08


In [10]:
df = pd.concat([df_label,df_data_mod], axis = 1)

In [11]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [12]:
train

,PLATINUM_STATUS,label,ENSG00000015413,ENSG00000048545,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
ID,,,,,,,,,,,,,,,,,,,,,
TCGA-23-1030,Sensitive,0.0,4.206460e-07,5.365826e-08,0.000014,5.892080e-08,1.141525e-06,0.000000e+00,0.000000e+00,2.400000e-08,...,0.000045,0.000004,0.000008,0.000000,0.000159,0.000000,1.043235e-06,0.000002,0.000267,0.000000e+00
TCGA-24-1434,Resistant,1.0,7.563406e-07,3.344640e-06,0.000008,7.117148e-06,1.480300e-05,4.211494e-08,4.376895e-08,4.031636e-07,...,0.000497,0.000000,0.000267,0.000013,0.000425,0.000000,0.000000e+00,0.000014,0.000786,8.338004e-09
TCGA-13-1403,Sensitive,0.0,8.019886e-05,1.547114e-07,0.000003,2.172565e-07,2.019678e-06,2.532518e-08,1.315990e-08,0.000000e+00,...,0.000000,0.000007,0.000002,0.000002,0.000003,0.000003,3.007932e-06,0.000002,0.000012,0.000000e+00
TCGA-10-0933,Sensitive,0.0,5.224658e-07,0.000000e+00,0.000273,2.251782e-08,2.850839e-06,1.437497e-07,0.000000e+00,2.120328e-06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000e+00,0.000015,0.000000,3.659126e-08
TCGA-04-1332,Sensitive,0.0,6.597888e-06,2.120660e-07,0.000003,6.493335e-08,6.152029e-07,0.000000e+00,5.411560e-08,8.782539e-07,...,0.000003,0.000003,0.000003,0.000000,0.000039,0.000004,9.792201e-06,0.000018,0.000050,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-36-1574,Sensitive,0.0,1.579900e-05,4.882248e-06,0.000074,1.918095e-08,2.854636e-06,3.122416e-06,0.000000e+00,1.016690e-07,...,0.000196,0.000000,0.000298,0.000036,0.000324,0.000004,0.000000e+00,0.000014,0.000637,0.000000e+00
TCGA-24-1474,Sensitive,0.0,1.774466e-06,9.054147e-08,0.000066,2.103143e-07,8.755350e-07,5.928402e-08,3.080616e-08,8.107459e-08,...,0.000005,0.000009,0.000000,0.000014,0.000000,0.000000,4.400815e-06,0.000012,0.000004,1.877948e-07
TCGA-24-2254,Sensitive,0.0,3.283081e-06,4.187950e-07,0.000062,2.210906e-08,9.719384e-06,5.484305e-07,1.603039e-07,2.812549e-07,...,0.000000,0.000000,0.000005,0.000000,0.000033,0.000000,2.035575e-06,0.000111,0.000031,2.714486e-08


In [13]:
train.to_csv("train_platin.csv", mode='w')

In [14]:
val

,PLATINUM_STATUS,label,ENSG00000015413,ENSG00000048545,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
ID,,,,,,,,,,,,,,,,,,,,,
TCGA-04-1536,Sensitive,0.0,6.505107e-07,0.000000e+00,8.110216e-06,1.557580e-08,7.132598e-08,0.000000e+00,6.274106e-09,0.000000e+00,...,0.000002,0.000000e+00,0.000004,0.000000,0.000009,0.000000,1.434060e-06,0.000003,0.000031,1.912352e-08
TCGA-25-1320,Sensitive,0.0,6.688145e-06,1.661788e-06,2.906911e-05,4.110742e-07,1.721730e-06,3.108836e-07,8.077329e-08,1.860000e-07,...,0.000941,2.851000e-06,0.002429,0.000019,0.003512,0.000002,2.307774e-06,0.000023,0.001185,0.000000e+00
TCGA-29-1766,Sensitive,0.0,3.531866e-07,1.441697e-07,7.445647e-06,1.917977e-07,2.648827e-06,0.000000e+00,3.678963e-08,1.613695e-07,...,0.000381,0.000000e+00,0.000139,0.000000,0.000847,0.000005,7.007445e-07,0.000000,0.000369,0.000000e+00
TCGA-13-0908,Sensitive,0.0,6.508341e-07,7.274258e-07,8.219628e-06,7.346530e-08,3.119515e-06,0.000000e+00,5.380479e-08,2.832032e-08,...,0.000143,3.038555e-06,0.000307,0.000007,0.000337,0.000000,9.223542e-07,0.000005,0.000154,0.000000e+00
TCGA-24-1562,Resistant,1.0,7.318205e-06,4.430000e-07,9.489463e-06,4.918231e-07,2.668363e-05,5.387235e-07,6.460168e-08,1.983525e-07,...,0.000074,0.000000e+00,0.000063,0.000010,0.000124,0.000002,0.000000e+00,0.000014,0.000663,0.000000e+00
TCGA-09-2045,Sensitive,0.0,2.145301e-06,7.300000e-08,5.349291e-06,1.818392e-07,2.822688e-07,2.389115e-07,0.000000e+00,0.000000e+00,...,0.000004,8.763813e-06,0.000011,0.000000,0.000083,0.000000,0.000000e+00,0.000019,0.000051,0.000000e+00
TCGA-24-1435,Sensitive,0.0,4.751326e-07,4.640892e-06,1.084729e-05,2.223305e-07,6.564154e-06,9.070825e-08,0.000000e+00,6.200000e-08,...,0.000004,1.663692e-06,0.000048,0.000007,0.000005,0.000000,6.733522e-07,0.000008,0.000093,9.000000e-09
TCGA-61-1918,Sensitive,0.0,7.252694e-06,9.026001e-08,1.924740e-05,3.430809e-08,1.745627e-06,2.954987e-07,1.074864e-07,4.041128e-07,...,0.000025,8.238065e-05,0.000000,0.000005,0.000004,0.000003,7.896843e-06,0.000023,0.000013,0.000000e+00
TCGA-24-2033,Resistant,1.0,1.518720e-06,1.963134e-06,1.032795e-06,4.363701e-09,6.794089e-06,0.000000e+00,7.030988e-08,0.000000e+00,...,0.000040,4.963323e-06,0.000005,0.000000,0.000040,0.000004,5.022060e-06,0.000009,0.000026,4.018219e-08


In [15]:
val.to_csv("val_platin.csv", mode='w')

In [4]:
train = pd.read_csv("train_platin.csv")
val = pd.read_csv("val_platin.csv")

In [5]:
trn_X_pd = train.iloc[:,3:]
trn_y_pd = train.iloc[:,2]

val_X_pd = val.iloc[:,3:]
val_y_pd = val.iloc[:,2]

In [6]:
val_X_pd

,ENSG00000015413,ENSG00000048545,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
0,6.505107e-07,0.000000e+00,8.110216e-06,1.557580e-08,7.132598e-08,0.000000e+00,6.274106e-09,0.000000e+00,3.506581e-07,0.000003,...,0.000002,0.000000e+00,0.000004,0.000000,0.000009,0.000000,1.434060e-06,0.000003,0.000031,1.912352e-08
1,6.688145e-06,1.661788e-06,2.906911e-05,4.110742e-07,1.721730e-06,3.108836e-07,8.077329e-08,1.860000e-07,5.643000e-06,0.000038,...,0.000941,2.851000e-06,0.002429,0.000019,0.003512,0.000002,2.307774e-06,0.000023,0.001185,0.000000e+00
2,3.531866e-07,1.441697e-07,7.445647e-06,1.917977e-07,2.648827e-06,0.000000e+00,3.678963e-08,1.613695e-07,1.507852e-06,0.000002,...,0.000381,0.000000e+00,0.000139,0.000000,0.000847,0.000005,7.007445e-07,0.000000,0.000369,0.000000e+00
3,6.508341e-07,7.274258e-07,8.219628e-06,7.346530e-08,3.119515e-06,0.000000e+00,5.380479e-08,2.832032e-08,4.029561e-06,0.000008,...,0.000143,3.038555e-06,0.000307,0.000007,0.000337,0.000000,9.223542e-07,0.000005,0.000154,0.000000e+00
4,7.318205e-06,4.430000e-07,9.489463e-06,4.918231e-07,2.668363e-05,5.387235e-07,6.460168e-08,1.983525e-07,3.610570e-06,0.000015,...,0.000074,0.000000e+00,0.000063,0.000010,0.000124,0.000002,0.000000e+00,0.000014,0.000663,0.000000e+00
5,2.145301e-06,7.300000e-08,5.349291e-06,1.818392e-07,2.822688e-07,2.389115e-07,0.000000e+00,0.000000e+00,2.775422e-07,0.000009,...,0.000004,8.763813e-06,0.000011,0.000000,0.000083,0.000000,0.000000e+00,0.000019,0.000051,0.000000e+00
6,4.751326e-07,4.640892e-06,1.084729e-05,2.223305e-07,6.564154e-06,9.070825e-08,0.000000e+00,6.200000e-08,2.766101e-06,0.000014,...,0.000004,1.663692e-06,0.000048,0.000007,0.000005,0.000000,6.733522e-07,0.000008,0.000093,9.000000e-09
7,7.252694e-06,9.026001e-08,1.924740e-05,3.430809e-08,1.745627e-06,2.954987e-07,1.074864e-07,4.041128e-07,1.544756e-06,0.000019,...,0.000025,8.238065e-05,0.000000,0.000005,0.000004,0.000003,7.896843e-06,0.000023,0.000013,0.000000e+00
8,1.518720e-06,1.963134e-06,1.032795e-06,4.363701e-09,6.794089e-06,0.000000e+00,7.030988e-08,0.000000e+00,4.912000e-06,0.000055,...,0.000040,4.963323e-06,0.000005,0.000000,0.000040,0.000004,5.022060e-06,0.000009,0.000026,4.018219e-08
9,2.306812e-06,2.140000e-07,1.235971e-05,1.054471e-07,1.517598e-06,1.588097e-06,3.640738e-08,1.022000e-06,1.220879e-06,0.000014,...,0.000512,3.940780e-05,0.000374,0.000019,0.002329,0.000003,0.000000e+00,0.000010,0.001907,6.473247e-08


In [7]:
trn_X_pd

,ENSG00000015413,ENSG00000048545,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
0,4.206460e-07,5.365826e-08,0.000014,5.892080e-08,1.141525e-06,0.000000e+00,0.000000e+00,2.400000e-08,9.183352e-07,0.000002,...,0.000045,0.000004,0.000008,0.000000,0.000159,0.000000,1.043235e-06,0.000002,0.000267,0.000000e+00
1,7.563406e-07,3.344640e-06,0.000008,7.117148e-06,1.480300e-05,4.211494e-08,4.376895e-08,4.031636e-07,2.935482e-06,0.000015,...,0.000497,0.000000,0.000267,0.000013,0.000425,0.000000,0.000000e+00,0.000014,0.000786,8.338004e-09
2,8.019886e-05,1.547114e-07,0.000003,2.172565e-07,2.019678e-06,2.532518e-08,1.315990e-08,0.000000e+00,6.693078e-06,0.000007,...,0.000000,0.000007,0.000002,0.000002,0.000003,0.000003,3.007932e-06,0.000002,0.000012,0.000000e+00
3,5.224658e-07,0.000000e+00,0.000273,2.251782e-08,2.850839e-06,1.437497e-07,0.000000e+00,2.120328e-06,4.890517e-06,0.000006,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000004,0.000000e+00,0.000015,0.000000,3.659126e-08
4,6.597888e-06,2.120660e-07,0.000003,6.493335e-08,6.152029e-07,0.000000e+00,5.411560e-08,8.782539e-07,3.831000e-06,0.000014,...,0.000003,0.000003,0.000003,0.000000,0.000039,0.000004,9.792201e-06,0.000018,0.000050,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,1.579900e-05,4.882248e-06,0.000074,1.918095e-08,2.854636e-06,3.122416e-06,0.000000e+00,1.016690e-07,4.318207e-07,0.000008,...,0.000196,0.000000,0.000298,0.000036,0.000324,0.000004,0.000000e+00,0.000014,0.000637,0.000000e+00
167,1.774466e-06,9.054147e-08,0.000066,2.103143e-07,8.755350e-07,5.928402e-08,3.080616e-08,8.107459e-08,5.165244e-07,0.000008,...,0.000005,0.000009,0.000000,0.000014,0.000000,0.000000,4.400815e-06,0.000012,0.000004,1.877948e-07
168,3.283081e-06,4.187950e-07,0.000062,2.210906e-08,9.719384e-06,5.484305e-07,1.603039e-07,2.812549e-07,1.194579e-06,0.000026,...,0.000000,0.000000,0.000005,0.000000,0.000033,0.000000,2.035575e-06,0.000111,0.000031,2.714486e-08
169,5.076250e-06,6.386642e-07,0.000005,6.000000e-09,5.764151e-05,9.292876e-08,1.207230e-08,2.446403e-06,1.619323e-06,0.000036,...,0.000000,0.000002,0.000000,0.000000,0.000006,0.000000,6.898357e-07,0.000003,0.000028,9.199116e-09


In [8]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

val_X = torch.from_numpy(val_X_pd.astype(float).values)
val_y = torch.from_numpy(val_y_pd.astype(float).values)

#trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
#trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

#val_X = torch.from_numpy(val_X_pd.astype(float).values)
#val_y = torch.from_numpy(val_y_pd.astype(float).values)

In [9]:
batch_size = 16

In [10]:
#trn_X = trn_X.float()
#val_X = val_X.float()

#trn = data_utils.TensorDataset(trn_X, trn_y)
#trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

#val = data_utils.TensorDataset(val_X, val_y)
#val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [11]:
#tmp = next(iter(trn_loader))

In [12]:
#tmp

In [13]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [14]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [15]:
tmp = next(iter(trn_loader))

In [16]:
tmp

{'X': tensor([[4.2868e-07, 2.8873e-06, 9.6202e-06,  ..., 1.4071e-05, 1.2171e-04,
          0.0000e+00],
         [8.6707e-06, 1.7290e-06, 1.2200e-05,  ..., 1.2582e-06, 4.2951e-03,
          0.0000e+00],
         [7.1443e-06, 4.3920e-07, 1.0624e-04,  ..., 1.0323e-04, 5.0276e-03,
          1.1387e-08],
         ...,
         [3.4584e-06, 6.6175e-07, 1.4944e-05,  ..., 0.0000e+00, 1.5562e-03,
          0.0000e+00],
         [6.0381e-06, 4.6600e-07, 2.0704e-06,  ..., 8.6478e-06, 1.4629e-02,
          0.0000e+00],
         [5.7607e-06, 1.1757e-06, 4.2440e-06,  ..., 6.0605e-06, 3.8943e-05,
          0.0000e+00]], dtype=torch.float64),
 'y': tensor([1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1.],
        dtype=torch.float64)}

In [17]:
num_batches = len(trn_loader)

## Build Model

In [18]:
use_cuda = torch.cuda.is_available()

In [19]:
class MLPRegressor(nn.Module):
    
    def __init__(self):
        super(MLPRegressor, self).__init__()
        h1 = nn.Linear((len(train.columns)-3), 32)
        h2 = nn.Linear(32, 16)
        h3 = nn.Linear(16, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if use_cuda:
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1)

In [20]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
         
        self.layer1 = nn.Sequential(
            torch.nn.Linear((len(train.columns)-3), 256, bias=True),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU()
        )
 
        self.layer2 = nn.Sequential(
            torch.nn.Linear(256, 128, bias=True),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU()
        )
    
        self.layer3 = nn.Sequential(
            torch.nn.Linear(128, 64, bias=True),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU()
        )
 
        self.layer4 = nn.Sequential(
            torch.nn.Linear(64, 10, bias=True),
            
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.layer1(x)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer2(x_out)
        x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer3(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer4(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return x_out
 


In [21]:
class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((len(train.columns)-3), 256, bias=True),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),

            torch.nn.Linear(256, 128, bias=True),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(128, 64, bias=True),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            
            torch.nn.Linear(64, 32, bias=True),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(32, 16, bias=True),
            torch.nn.BatchNorm1d(16),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(16, 1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [22]:
class DNN_seq_(nn.Module):
    def __init__(self):
        super(DNN_seq_, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((len(train.columns)-3), 256, bias=True),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(
            torch.nn.Linear(256, 128, bias=True),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            
            torch.nn.Linear(128, 64, bias=True),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            
            torch.nn.Linear(64, 32, bias=True),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(32,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [23]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight) 

In [24]:
model = DNN_seq()

In [25]:
model.apply(weights_init)

DNN_seq(
  (classifier): Sequential(
    (0): Linear(in_features=397, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
  )
  (classifier3): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Batch

## Train model

In [28]:
model = DNN_seq()

In [29]:
criterion = nn.BCELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
num_epochs = 20
num_batches = len(trn_loader) # requier modification

In [31]:
num_batches

11

## Train 1

In [42]:
trn_loss_list = []
val_loss_list = []

for epoch in range(num_epochs):
    trn_loss_summary = 0.0
    correct = 0   # Accuracy

    for i, trn in enumerate(trn_loader):
        trn_X, trn_y = trn['X'], trn['y']
        if use_cuda:
            trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
        trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
        optimizer.zero_grad()
        trn_pred = model(trn_X)
        #trn_pred = trn_pred.type()
        #type_as(torch.cuda.IntTensor())
        trn_loss = criterion(trn_pred, trn_y)
        trn_loss.backward()
        optimizer.step()
        
        trn_loss_summary += trn_loss
        
        predicted = torch.max(trn_pred.data, 1)[1] 
        correct += (predicted == trn_y).sum()
        if (i+1) % 5 == 0:
            correct_val = 0
            with torch.no_grad():
                val_loss_summary = 0.0

                for j, val in enumerate(val_loader):
                    val_X, val_y = val['X'], val['y']
                    if use_cuda:
                        val_X, val_y = val_X.cuda(), val_y.cuda()
                    val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
                    val_pred = model(val_X)
                    val_pred = val_pred.type_as(torch.FloatTensor())
                    val_loss = criterion(val_pred, val_y)
                    val_loss_summary += val_loss
                    predicted = torch.max(val_pred.data, 1)[1] 
                    correct_val += (predicted == val_y).sum()
                
            print("epoch: {}/{} | step: {}/{} | trn_loss: {:.4f} | val_loss: {:.4f}".format(
                epoch + 1, num_epochs, i+1, num_batches, (trn_loss_summary/5)**(1/2), (val_loss_summary/len(val_loader))**(1/2)
            ))
            print("Train_accuracy: {:.3f}".format(float(correct*100) / float(batch_size*(i+1))))
            print("Val_accuracy: {:.3f}".format(float(correct_val) / float(batch_size*(i+1))))
            trn_loss_list.append((trn_loss_summary/5)**(1/2))
            val_loss_list.append((val_loss_summary/len(val_loader))**(1/2))
            trn_loss_summary = 0.0
        
print("finish Training")

/home/dbqls/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/home/dbqls/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch: 1/20 | step: 5/11 | trn_loss: 0.8871 | val_loss: 0.9068
Train_accuracy: 60.000
Val_accuracy: 0.438
epoch: 1/20 | step: 10/11 | trn_loss: 0.8805 | val_loss: 0.8943
Train_accuracy: 66.250
Val_accuracy: 0.219
epoch: 2/20 | step: 5/11 | trn_loss: 0.8590 | val_loss: 0.8730
Train_accuracy: 75.000
Val_accuracy: 0.438
epoch: 2/20 | step: 10/11 | trn_loss: 0.8440 | val_loss: 0.8820
Train_accuracy: 68.750
Val_accuracy: 0.219
epoch: 3/20 | step: 5/11 | trn_loss: 0.8613 | val_loss: 0.8336
Train_accuracy: 63.750
Val_accuracy: 0.438
epoch: 3/20 | step: 10/11 | trn_loss: 0.8834 | val_loss: 0.8705
Train_accuracy: 66.875
Val_accuracy: 0.219
epoch: 4/20 | step: 5/11 | trn_loss: 0.8437 | val_loss: 0.8430
Train_accuracy: 68.750
Val_accuracy: 0.438
epoch: 4/20 | step: 10/11 | trn_loss: 0.8178 | val_loss: 0.8681
Train_accuracy: 66.875
Val_accuracy: 0.219
epoch: 5/20 | step: 5/11 | trn_loss: 0.8545 | val_loss: 0.8481
Train_accuracy: 70.000
Val_accuracy: 0.438
epoch: 5/20 | step: 10/11 | trn_loss: 0.84

# Train_2

In [38]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)#,lr=0.001, betas=(0.9,0.999))
    error = nn.BCELoss()
    num_epochs = 10    
    
    model.train()
    for epoch in range(num_epochs):
        trn_loss_summary = 0.0
        correct = 0   # Accuracy

        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)
            #trn_pred = trn_pred.type()
            #type_as(torch.cuda.IntTensor())
            trn_loss = criterion(trn_pred, trn_y)
            trn_loss.backward()
            optimizer.step()
        
            trn_loss_summary += trn_loss
        
            predicted = torch.max(trn_pred.data, 1)[1] 
            correct += (predicted == trn_y).sum()
            #print(correct)
            if batch_idx % 2 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(trn_X), len(trn_loader.dataset), 100.*batch_idx / len(trn_loader), trn_loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))

In [39]:
model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.664526	 Accuracy:62.500%
Epoch : 0 [32/171 (18%)]	Loss: 0.637009	 Accuracy:64.583%
Epoch : 0 [64/171 (36%)]	Loss: 0.662163	 Accuracy:65.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.701038	 Accuracy:66.964%
Epoch : 0 [128/171 (73%)]	Loss: 0.665930	 Accuracy:66.667%
Epoch : 0 [110/171 (91%)]	Loss: 0.652026	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.867415	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.733626	 Accuracy:60.417%
Epoch : 1 [64/171 (36%)]	Loss: 0.603864	 Accuracy:72.500%
Epoch : 1 [96/171 (55%)]	Loss: 0.630444	 Accuracy:72.321%
Epoch : 1 [128/171 (73%)]	Loss: 0.781552	 Accuracy:69.444%
Epoch : 1 [110/171 (91%)]	Loss: 0.740844	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.707396	 Accuracy:43.750%
Epoch : 2 [32/171 (18%)]	Loss: 0.665379	 Accuracy:62.500%
Epoch : 2 [64/171 (36%)]	Loss: 0.523821	 Accuracy:63.750%


/home/dbqls/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch : 2 [96/171 (55%)]	Loss: 0.743549	 Accuracy:67.857%
Epoch : 2 [128/171 (73%)]	Loss: 0.593251	 Accuracy:67.361%
Epoch : 2 [110/171 (91%)]	Loss: 0.557755	 Accuracy:65.909%
Epoch : 3 [0/171 (0%)]	Loss: 0.722250	 Accuracy:62.500%
Epoch : 3 [32/171 (18%)]	Loss: 0.788930	 Accuracy:54.167%
Epoch : 3 [64/171 (36%)]	Loss: 0.572536	 Accuracy:62.500%
Epoch : 3 [96/171 (55%)]	Loss: 0.564072	 Accuracy:64.286%
Epoch : 3 [128/171 (73%)]	Loss: 0.498964	 Accuracy:67.361%
Epoch : 3 [110/171 (91%)]	Loss: 0.567501	 Accuracy:65.909%
Epoch : 4 [0/171 (0%)]	Loss: 0.512342	 Accuracy:81.250%
Epoch : 4 [32/171 (18%)]	Loss: 0.742657	 Accuracy:66.667%
Epoch : 4 [64/171 (36%)]	Loss: 0.641298	 Accuracy:65.000%
Epoch : 4 [96/171 (55%)]	Loss: 0.544529	 Accuracy:66.071%
Epoch : 4 [128/171 (73%)]	Loss: 0.508471	 Accuracy:65.972%
Epoch : 4 [110/171 (91%)]	Loss: 0.729375	 Accuracy:65.909%
Epoch : 5 [0/171 (0%)]	Loss: 0.609767	 Accuracy:62.500%
Epoch : 5 [32/171 (18%)]	Loss: 0.627082	 Accuracy:66.667%
Epoch : 5 [64/

In [40]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5850959420204163, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


# Visualization

In [41]:
plt.figure(figsize=(16,9))
x_range = range(len(trn_loss_list))
plt.plot(x_range, trn_loss_list, label="trn")
plt.plot(x_range, val_loss_list, label="val")
plt.legend()
plt.xlabel("training steps")
plt.ylabel("loss")

NameError: name 'trn_loss_list' is not defined

<Figure size 1152x648 with 0 Axes>

## Transfer Learning

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.
실질적 조언
새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우

데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.
새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우

새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.
새로 훈련할 데이터가 적으며 original 데이터와 다른 경우

데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.
새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우

데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

In [103]:
model.classifier3

Sequential(
  (0): Linear(in_features=32, out_features=16, bias=True)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.5, inplace=False)
)

In [149]:
model_2 = DNN_seq()

In [150]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = model_2.classifier    
model.classifier2 = model_2.classifier2


In [151]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.746858	 Accuracy:50.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.662493	 Accuracy:60.417%
Epoch : 0 [64/171 (36%)]	Loss: 0.704260	 Accuracy:60.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.674394	 Accuracy:66.071%
Epoch : 0 [128/171 (73%)]	Loss: 0.722818	 Accuracy:67.361%
Epoch : 0 [110/171 (91%)]	Loss: 0.599336	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.575366	 Accuracy:87.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.667310	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.673750	 Accuracy:68.750%
Epoch : 1 [96/171 (55%)]	Loss: 0.751240	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.740712	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.710288	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.641428	 Accuracy:62.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.618767	 Accuracy:68.750%
Epoch : 2 [64/171 (36%)]	Loss: 0.619703	 Accuracy:65.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.579154	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.667362	 Accuracy:66.667%
Epoch : 2 [110/

In [152]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5934396982192993, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


## Voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier()),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('lr', LogisticRegressionCV()),
    ('ridge', RidgeClassifier()),
]

# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote_cv = cross_validate(hard_vote, x_train, y_train, cv=k_fold)
hard_vote.fit(x_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote_cv = cross_validate(soft_vote, x_train, y_train, cv=k_fold)
soft_vote.fit(x_train, y_train)


In [42]:
model_1 = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_1 = model_1.cuda()
    
fit(model_1, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.648584	 Accuracy:75.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.685633	 Accuracy:62.500%
Epoch : 0 [64/171 (36%)]	Loss: 0.738034	 Accuracy:68.750%
Epoch : 0 [96/171 (55%)]	Loss: 0.718815	 Accuracy:65.179%
Epoch : 0 [128/171 (73%)]	Loss: 0.663025	 Accuracy:68.056%
Epoch : 0 [110/171 (91%)]	Loss: 0.574838	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.597974	 Accuracy:75.000%
Epoch : 1 [32/171 (18%)]	Loss: 0.765819	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.666601	 Accuracy:67.500%
Epoch : 1 [96/171 (55%)]	Loss: 0.680634	 Accuracy:66.964%
Epoch : 1 [128/171 (73%)]	Loss: 0.619182	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.686758	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.815342	 Accuracy:50.000%
Epoch : 2 [32/171 (18%)]	Loss: 0.705732	 Accuracy:52.083%
Epoch : 2 [64/171 (36%)]	Loss: 0.622576	 Accuracy:60.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.629287	 Accuracy:63.393%
Epoch : 2 [128/171 (73%)]	Loss: 0.515706	 Accuracy:68.750%
Epoch : 2 [110/

In [46]:
model_2 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_2 = model_2.cuda()
    
fit(model_2, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.726548	 Accuracy:56.250%
Epoch : 0 [32/171 (18%)]	Loss: 0.703508	 Accuracy:56.250%
Epoch : 0 [64/171 (36%)]	Loss: 0.612747	 Accuracy:61.250%
Epoch : 0 [96/171 (55%)]	Loss: 0.575566	 Accuracy:65.179%
Epoch : 0 [128/171 (73%)]	Loss: 0.544716	 Accuracy:65.972%
Epoch : 0 [110/171 (91%)]	Loss: 0.611560	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.641374	 Accuracy:56.250%
Epoch : 1 [32/171 (18%)]	Loss: 0.633138	 Accuracy:64.583%
Epoch : 1 [64/171 (36%)]	Loss: 0.616448	 Accuracy:67.500%
Epoch : 1 [96/171 (55%)]	Loss: 0.550823	 Accuracy:69.643%
Epoch : 1 [128/171 (73%)]	Loss: 0.718642	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.496391	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.498467	 Accuracy:87.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.887428	 Accuracy:70.833%
Epoch : 2 [64/171 (36%)]	Loss: 0.580738	 Accuracy:68.750%
Epoch : 2 [96/171 (55%)]	Loss: 0.500280	 Accuracy:71.429%
Epoch : 2 [128/171 (73%)]	Loss: 0.805142	 Accuracy:67.361%
Epoch : 2 [110/

In [47]:
model_1.eval()
predict_1 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_1(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_1.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.579935610294342, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [48]:
model_2.eval()
predict_2 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_2(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_2.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.6875712871551514, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [49]:
predict_1

[array([0.4072227], dtype=float32),
 array([0.34137], dtype=float32),
 array([0.35161027], dtype=float32),
 array([0.37793928], dtype=float32),
 array([0.40149003], dtype=float32),
 array([0.40974772], dtype=float32),
 array([0.40991527], dtype=float32),
 array([0.40998164], dtype=float32),
 array([0.40864408], dtype=float32),
 array([0.34985468], dtype=float32),
 array([0.41095164], dtype=float32),
 array([0.407063], dtype=float32),
 array([0.41152972], dtype=float32),
 array([0.40827367], dtype=float32),
 array([0.4053927], dtype=float32),
 array([0.39758298], dtype=float32),
 array([0.41030633], dtype=float32),
 array([0.3969093], dtype=float32),
 array([0.41018578], dtype=float32),
 array([0.40626967], dtype=float32),
 array([0.36673126], dtype=float32),
 array([0.41220325], dtype=float32),
 array([0.41033667], dtype=float32),
 array([0.40302122], dtype=float32),
 array([0.41204548], dtype=float32),
 array([0.4114596], dtype=float32),
 array([0.4024302], dtype=float32),
 array([0.4

In [50]:
predict_2

[array([0.46569666], dtype=float32),
 array([0.34076282], dtype=float32),
 array([0.4635323], dtype=float32),
 array([0.42451888], dtype=float32),
 array([0.4830429], dtype=float32),
 array([0.4781332], dtype=float32),
 array([0.42165685], dtype=float32),
 array([0.4209946], dtype=float32),
 array([0.47860295], dtype=float32),
 array([0.36543727], dtype=float32),
 array([0.46133387], dtype=float32),
 array([0.5047384], dtype=float32),
 array([0.48215577], dtype=float32),
 array([0.43285683], dtype=float32),
 array([0.4470338], dtype=float32),
 array([0.4535735], dtype=float32),
 array([0.4250045], dtype=float32),
 array([0.51857483], dtype=float32),
 array([0.46541592], dtype=float32),
 array([0.45539773], dtype=float32),
 array([0.4434651], dtype=float32),
 array([0.4886103], dtype=float32),
 array([0.3854558], dtype=float32),
 array([0.4322617], dtype=float32),
 array([0.5380801], dtype=float32),
 array([0.45171666], dtype=float32),
 array([0.47342297], dtype=float32),
 array([0.5035

In [52]:
#predict_3 = predict_1 + predict_2

In [54]:
len(predict_3)

86

In [56]:
type(predict_1)

list

In [92]:
import operator

predict_3 = map(operator.add, predict_1, predict_2)

In [93]:
print(predict_3)

In [94]:
predict = []

for i in predict_3:
    print (i/2)
    predict.append(i/2)
    print (predict)



[0.43645966]
[array([0.43645966], dtype=float32)]
[0.34106642]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32)]
[0.4075713]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32)]
[0.40122908]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32)]
[0.44226646]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32)]
[0.44394046]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32)]
[0.41578606]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0

In [97]:
print(predict)
type(predict)

[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32), array([0.41578606], dtype=float32), array([0.41548812], dtype=float32), array([0.4436235], dtype=float32), array([0.357646], dtype=float32), array([0.43614274], dtype=float32), array([0.4559007], dtype=float32), array([0.44684273], dtype=float32), array([0.42056525], dtype=float32), array([0.42621326], dtype=float32), array([0.42557824], dtype=float32), array([0.4176554], dtype=float32), array([0.45774207], dtype=float32), array([0.43780085], dtype=float32), array([0.4308337], dtype=float32), array([0.4050982], dtype=float32), array([0.4504068], dtype=float32), array([0.39789623], dtype=float32), array([0.41764146], dtype=float32), array([0.4750628], dtype=float32), array([0.4315881], dtype=float32), array([0.4379266], dtype=float32), array([0.4739846], dtype=float32), arra

list

In [104]:
val_y_pd_ = val_y_pd.to_numpy()
val_y_pd_ = torch.from_numpy(val_y_pd_)
val_y_pd_ = val_y_pd_.type_as(torch.FloatTensor())

In [ ]:
val_loss = criterion(predict, val_y)
val_loss_summary += val_loss
predicted = torch.max(val_pred.data, 1)[1] 
correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

In [96]:
val_y_pd

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    1.0
27    0.0
28    0.0
29    1.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
Name: label, dtype: float64

In [93]:
#num_ftrs = model.forward.in_features
 
for name, param in model.named_parameters():
    print (name, param.data)

layer1.0.weight tensor([[-0.0565, -0.1205,  0.0296,  ...,  0.2230, -0.0025,  0.0114],
        [-0.0879, -0.0202, -0.0235,  ...,  0.0587,  0.0370, -0.0111],
        [-0.0381, -0.0511, -0.0141,  ..., -0.0142,  0.0329,  0.0550],
        ...,
        [ 0.0349, -0.0490, -0.0410,  ..., -0.0243, -0.0495,  0.0048],
        [ 0.0487,  0.1016, -0.0085,  ..., -0.1036,  0.0225, -0.0505],
        [-0.0129,  0.1087,  0.0454,  ..., -0.1197,  0.0375,  0.0101]])
layer1.0.bias tensor([-0.0296, -0.0339,  0.0066, -0.0166, -0.0347, -0.0489,  0.0233,  0.0351,
         0.0348,  0.0116,  0.0404, -0.0197, -0.0154,  0.0126,  0.0236,  0.0361,
        -0.0178, -0.0001, -0.0028, -0.0274, -0.0495, -0.0317,  0.0020,  0.0040,
         0.0019,  0.0077,  0.0134, -0.0460,  0.0494,  0.0400, -0.0072, -0.0119,
         0.0044, -0.0115,  0.0237, -0.0051, -0.0136, -0.0101,  0.0195,  0.0419,
        -0.0333, -0.0286,  0.0231, -0.0157,  0.0131,  0.0034, -0.0038,  0.0113,
        -0.0342,  0.0022, -0.0403,  0.0045,  0.0368, -0.

In [95]:
model.layer1 = 

NameError: name 'Sequential' is not defined

In [79]:
nn.Linear(num_ftrs)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [32]:
list(model.children())[0:3]

AttributeError: 'list' object has no attribute 'parameter'

## Train model

In [77]:
model = MLPRegressor()

NameError: name 'X_features' is not defined

In [78]:
criterion = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [44]:
batch_size = 32

In [45]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = data_utils.TensorDataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

NameError: name 'val_X' is not defined

In [13]:
trn_X_pd = df2_mod.iloc[:,2:]

In [14]:
trn_X_pd

ID,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,ENSG00000109321,ENSG00000112499,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,0.970585,0.003561,0.044476,0.000000,0.000000,0.000000,0.043732,2.308592,1.014494,0.000000,...,0.000000,0.920597,0.000000,1.604301,0.299376,0.000000,0.894232,1.612154,0.761720,0.000000
TCGA-24-0982,0.395276,0.005010,0.090384,0.007062,0.000000,0.006438,0.082034,0.242661,0.691196,0.004156,...,24.797152,0.259035,4.625172,0.188089,4.211878,0.270091,0.034947,0.311310,0.357217,0.000000
TCGA-36-1580,1.403594,0.008641,0.133845,0.001970,0.002559,0.041762,0.074384,0.984290,2.329158,0.001391,...,4.017149,0.289083,0.893372,0.314861,4.817979,0.000000,0.000000,0.707253,15.996040,0.000000
TCGA-25-1321,0.308600,0.003129,0.004478,0.162223,0.000788,0.000000,0.101272,0.240456,1.740308,0.004819,...,2.576079,0.166843,0.000000,0.121147,1.356422,0.521892,0.157563,2.033770,1.495528,0.000000
TCGA-30-1892,0.206074,0.133216,0.119614,0.056072,0.000000,0.055381,0.280458,0.592872,0.914734,0.004400,...,1.058604,0.114269,2.825069,0.000000,3.716018,0.000000,0.000000,0.294278,1.890974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,0.319288,0.000491,0.033696,0.005704,0.000988,0.011701,0.060742,0.663423,0.094262,0.000671,...,0.000000,0.000000,0.000000,3.038608,0.567030,0.000000,0.000000,0.071847,4.616723,0.000000
TCGA-24-0968,0.237050,0.009708,0.185222,0.000000,0.000000,0.000000,0.307004,0.213121,1.291373,0.003796,...,15.221570,0.525782,4.332951,2.720169,31.845646,0.205584,0.000000,0.710875,6.117782,0.000000
TCGA-30-1891,0.772034,0.000481,0.110194,0.000000,0.000646,0.200678,0.000000,0.098616,2.959271,0.077290,...,47.331214,0.000000,2.631492,0.000000,65.568652,0.000000,0.000000,0.000000,80.395413,0.000000
TCGA-09-0366,0.195435,0.000085,1.631170,0.026487,0.001376,0.006339,0.303848,0.703636,5.424154,0.008418,...,0.225023,0.534375,0.100086,1.481522,3.712525,0.000000,0.058985,2.477118,0.602935,0.001573


In [15]:
trn_y_pd = df2_mod.iloc[:,1]

In [16]:
trn_y_pd

TCGA-61-1910    0.007666
TCGA-24-0982    0.007190
TCGA-36-1580    0.027081
TCGA-25-1321    0.002315
TCGA-30-1892    0.057091
                  ...   
TCGA-04-1347    0.000000
TCGA-24-0968    0.013682
TCGA-30-1891    0.034186
TCGA-09-0366    0.032361
TCGA-13-0884    0.018597
Name: ENSG00000048545, Length: 214, dtype: float64

In [17]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

In [18]:
batch_size=64

In [19]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

In [20]:
tmp = next(iter(trn_loader))

In [21]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)

In [22]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)